# 决策树介绍
## 在做决策树的时候，会经历两个阶段，构造和剪枝
### 构造：选择什么属性作为节点
* 根节点：树的最顶端的节点
* 子节点：树中间的节点
* 叶节点：树末梢的节点

### 剪枝：决策树构造之后需要剪枝，防止过拟合。
* 预剪枝：在决策树构造时就进行剪枝。在构造过程中对节点进行判断，如果该节点不能给验证数据带来精度提升，就把该节点作为叶节点，不再划分下去。
* 后剪枝：生成决策树之后进行剪枝。通常从叶节点开始向上对每个节点进行判断。如果该节点子树不能给验证数据带来精度提升就剪掉，用这个节点子树的叶子节点代替该节点。


## 纯度和信息熵
### 纯度：把决策树的构造过程理解为寻找纯净划分的过程，换句话说纯度就是让目标变量的分歧最小
* 例：这里有3个集合，集合1是6次都打篮球；集合2是4次打，2次不打；集合3是3次打，3次不打。
* 按照纯度指标来说，集合1>集合2>集合3。因为集合1的分歧最小，集合3的分歧最大。

### 信息熵：反映信息的不确定度
* 当不确定性越大时，所包含的信息量越大，信息熵也越高。
* 例：5次打篮球，1次不打
* 分析：例子中有6次决策，打篮球5次，不打篮球1次。假设类别1为打篮球，类别2为不打篮球，节点划分为类别1的概率为5/6，为类别2的概率为1/6，代入计算过程，以log2为底
* 计算过程：Entropy(t) = -(1/6)*log2(1/6) - (5/6)*log2(5/6)

## 构造决策树的算法
### ID3算法
* 计算的是信息增益，该方式划分可带来纯度的提高，信息熵的下降
* 信息增益 = 父节点的信息熵 - 该父节点下的所有子节点的归一化信息熵
* 将信息增益最大的的节点作为父节点，可以得到纯度高的决策树


* 例：在某属性条件a下，父节点D是5次打篮球，5次不打；子节点D1是2次打篮球，1次不打；子节点D2是3次打篮球，4次不打。
* 计算过程：信息增益Gain(D,a) = 父节点信息熵Ent(D) - (3/10*Ent(D1) + 7/10*Ent(D2))

#### 缺点：倾向于选择取值比较多的属性，如果存在编号属性，那么编号也会被选为最优属性。所以有了C4.5算法

### C4.5算法
* 针对数据集有缺失值，也能进行处理。
* 比如ID3的例子中有10个样本数据，但现在缺了1个数据，只有9条样本，那么信息增益率先按照9条数据计算Gain，再用9/10*Gain计算得出信息增益。

### ID3和C4.5小结
* ID3算法优点是方法简单，缺点是对噪声敏感，如果训练数据有少量错误，可能会产生决策树分类错误。
* C4.5在ID3的基础上，使用信息增益率代替信息增益，解决噪声敏感问题。但需要对数据集进行多次扫描，算法效率相对较低。

### CART算法
#### 全称：Classification And Regression Tree，分类回归树。ID3和C4.5算法可生成二叉树或多叉树，而CART只支持二叉树。
* 分类树：处理离散数据，输出样本的类别，比如男、女，因为是从性别中来做选择。
* 回归树：处理连续数据，输出某区间的值，比如[18,28]之间的任意一个值。
* 采用基尼系数进行分类，基尼系数越小，不确定性越小，纯度越高。

## 使用CART算法创建分类树
### sklearn中自带的决策树分类器 DecisionTreeClassifier（criterion = ‘gini’）
** sklearn中只实现了ID3与CART决策树，所以在构造DecisionTreeClassifier类时，criterion意为标准。

** criterion = 'entropy',代表采用ID3分类树

** criterion = 'gini',代表采用CART分类树

In [7]:
from sklearn.model_selection import train_test_split   # 把数据划分为训练集和测试集
from sklearn.metrics import accuracy_score             # 评估模型的预测精度
from sklearn.tree import DecisionTreeClassifier        # CART分类树
from sklearn.datasets import load_iris                 # sklearn自带数据集
import pandas as pd
from IPython.display import display

In [8]:
# 给iris数据集构造一颗分类决策树
# 准备数据集
iris = load_iris()

# 查看数据集
# 构建dataframe时，每个key会变成一个列，values会变为行，每个values中的list长度不一致就会出现这个错误
# dn = pd.DataFrame(iris)   #arrays must all be same length
# 给iris加个中括号[]，能把每个key对应的value看成是一个值
df = pd.DataFrame([iris])
display(df.head())

# 获取特征集合、分类标识
features = iris.data
labels = iris.target

# 随机抽取33%的数据作为测试集，其余为训练集
train_features,test_features,train_labels,test_labels = train_test_split(features,labels,test_size = 0.33)    

# 创建 CART 分类树
clf = DecisionTreeClassifier(criterion = 'gini') # 初始化一颗CART分类树，使用gini系数对分类树进行训练

# 拟合构造CART分类树
clf = clf.fit(train_features,train_labels) # 将训练集的特征值和分类标识作为参数进行拟合

# 用CART分类树做预测
test_predict = clf.predict(test_features) # 传入测试集的特征值进行预测
print(test_predict)
print(test_labels)

# 预测结果与测试结果进行对比
score = accuracy_score(test_labels,test_predict) # 传入测试集的实际结果与预测值的结果进行对比
print(score)


,DESCR,data,feature_names,filename,target,target_names
0,.. _iris_dataset:\n\nIris plants dataset\n----...,"[[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2], [...","[sepal length (cm), sepal width (cm), petal le...",F:\Program Files (x86)\Anaconda3\lib\site-pack...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[setosa, versicolor, virginica]"


[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 1 2 1 2]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 2 2 1 2]
0.98


## 使用CART回归树做回归预测
** 使用sklearn自带的波士顿房价数据集，该数据集给出了影响房价的一些指标，比如犯罪率，房产税等，最后给出房价。

** 根据这些指标，使用CART回归树对波士顿房价进行预测

In [10]:
from sklearn.metrics import mean_squared_error   # mean_squared_error用于计算均方误差
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error  # 回归评价指标
from sklearn.model_selection import train_test_split  # 将数据分为测试集和训练集
from sklearn.tree import DecisionTreeRegressor   # CART预测树
from sklearn.datasets import load_boston         # 波士顿房价数据集

In [13]:
# 准备数据集
boston = load_boston()

# 探索数据
print(boston.feature_names)
dn2 = pd.DataFrame([boston])
display(dn2)

# 获取特征值和房价
features = boston.data
prices = boston.target

# 随机抽取33%的数据集作为测试集，其余为训练集
train_features,test_features,train_prices,test_prices = train_test_split(features,prices,test_size = 0.33)

# 创建CART回归树
dtr = DecisionTreeRegressor() # 初始化一颗CART回归树

# 拟合构造的CART回归树
dtr.fit(train_features,train_prices) # 将训练集的特征值和结果作为参数进行拟合

# 预测测试集中的房价
predict_prices = dtr.predict(test_features) # 传入测试集的特征值，得到预测房价
#print(predict_prices)
#print(test_prices)

# 预测树是处理连续的数据，不支持使用
#score2 = accuracy_score(test_features,predict_prices) #continuous is not supported 不支持连续数据

# 测试集的结果评价

print('回归树二乘偏差均值：',mean_squared_error(test_prices,predict_prices)) # 均方误差MSE越小，预测精度越高
print('回归树绝对值偏差均值：',mean_absolute_error(test_prices,predict_prices)) # 平均绝对误差，反映预测值误差的实际情况

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


,DESCR,data,feature_names,filename,target
0,.. _boston_dataset:\n\nBoston house prices dat...,"[[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2...","[CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD...",F:\Program Files (x86)\Anaconda3\lib\site-pack...,"[24.0, 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27...."


回归树二乘偏差均值： 27.552994012
回归树绝对值偏差均值： 3.43413173653
